In [ ]:
##########################################Loading Data################################################################

In [33]:
#importing libraries 
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve,auc
from sklearn.cluster import KMeans
from nltk.stem.porter import PorterStemmer 

import re 
import string 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem.wordnet import WordNetLemmatizer 
import pickle

In [34]:
#using the mythril python library for disassembly 
#https://github.com/ConsenSys/mythril
#takes bytecode and ouputs machine level opcodes 

from mythril.ether import asm,util
import os
import json
import logging


class Disassembly:

    def __init__(self, code):
        self.instruction_list = asm.disassemble(util.safe_decode(code))
        self.xrefs = []
        self.func_to_addr = {}
        self.addr_to_func = {}

        try:
            mythril_dir = os.environ['MYTHRIL_DIR']
        except KeyError:
            mythril_dir = os.path.join(os.path.expanduser('~'), ".mythril")

        # Load function signatures

        signatures_file = os.path.join(mythril_dir, 'signatures.json')

        if not os.path.exists(signatures_file):
            logging.info("Missing function signature file. Resolving of function names disabled.")
            signatures = {}
        else:
            with open(signatures_file) as f:
                signatures = json.load(f)

        # Parse jump table & resolve function names

        jmptable_indices = asm.find_opcode_sequence(["PUSH4", "EQ"], self.instruction_list)

        for i in jmptable_indices:
            func_hash = self.instruction_list[i]['argument']
            try:
                func_name = signatures[func_hash]
            except KeyError:
                func_name = "_function_" + func_hash

            try:
                offset = self.instruction_list[i+2]['argument']
                jump_target = int(offset, 16)

                self.func_to_addr[func_name] = jump_target
                self.addr_to_func[jump_target] = func_name
            except:
                continue



    def get_easm(self):

        return asm.instruction_list_to_easm(self.instruction_list)

In [35]:
#source Ethereum yellow paper 
EVM_OPCODE_SET = {'STOP', 'ADD', 'MUL', 'SUB', 'DIV', 'SDIV', 'MOD', 'SMOD', 'ADDMOD', 'MULMOD', 'EXP', 'SIGNEXTEND',
                 'LT','GT', 'SLT', 'SGT', 'EQ', 'ISZERO', 'AND', 'OR', 'XOR', 'NOT', 'BYTE','SHA3',
                 'ADDRESS', 'BALANCE', 'ORIGIN', 'CALLER', 'CALLVALUE', 'CALLDATALOAD', 'CALLDATASIZE', 'CALLDATACOPY', 
                  'CODESIZE', 'CODECOPY', 'GASPRICE','EXTCODESIZE','EXTCODECOPY', 'RETURNDATASIZE', 'RETURNDATACOPY',
                 'BLOCKHASH', 'COINBASE', 'TIMESTAMP', 'NUMBER', 'DIFFICULTY', 'GASLIMIT','POP','MLOAD', 'MSTORE', 'MSTORE8'
                 'SLOAD', 'SSTORE', 'JUMP', 'JUMPI', 'PC', 'MSIZE', 'GAS', 'JUMPDEST','PUSH1','PUSH2','PUSH3','PUSH4',
                  'PUSH5','PUSH6','PUSH7','PUSH8','PUSH9','PUSH10','PUSH11','PUSH12','PUSH13','PUSH14','PUSH15','PUSH16',
                 'PUSH17','PUSH18','PUSH19','PUSH20','PUSH21','PUSH22','PUSH23','PUSH24','PUSH25','PUSH26','PUSH27',
                 'PUSH28','PUSH29','PUSH30','PUSH31','PUSH32','DUP1','DUP2','DUP3','DUP4','DUP5','DUP6','DUP7','DUP8','DUP9',
                 'DUP10','DUP11','DUP12','DUP13','DUP14','DUP15','DUP16','DUP17','DUP18','SWAP1','SWAP2','SWAP3','SWAP4',
                 'SWAP5','SWAP6','SWAP7','SWAP8','SWAP9','SWAP10','SWAP11','SWAP12','SWAP13','SWAP14','SWAP15','SWAP16',
                 'LOG0','LOG1','LOG2','LOG3','LOG4','CREATE', 'CAL', 'CALLCODE','RETURN','DELEGATECALL' ,'STATICCALL','REVERT', 
                  'INVALID','SELFDESTRUCT'}

In [36]:
#######################################################################################################################
######################################################ORIGNAL_DATA#####################################################
#######################################################################################################################

In [37]:
#loading data
df = pd.read_csv('/Users/ravishaggarwal/Desktop/contracts.csv',sep = ',')

In [38]:
#bytecodes that are to be translated to opcodes 
bytecode = df['result.code']

In [39]:
#getting the list of opcodes translated from bytecodes 
opcode = (list(map(lambda x: Disassembly(x).get_easm(),bytecode)))

KeyboardInterrupt: 

In [ ]:
#converting opcodes list to dataframe
opcode_df = pd.DataFrame(opcode)

In [ ]:
#naming the opcodes dataframe
opcode_df.columns = ['opcode']

In [ ]:
#adding the opcodes data frame into the original data frame 
df['opcode'] = opcode_df

In [ ]:
#selecting the bytecode and the translated opcodes 
new_df = df[df.columns[3:]]
#new_df = df.columns['result.code','opcode']

In [ ]:
#new_df is the new dataframe consist of translated opcodes 
new_df.head(5)

In [40]:
#################################################CSV_WRITE############################################################
#writing the result into the csv file for future usage
#new_df.to_csv('new_data.csv', encoding='utf-8', index=False)

In [41]:
#clean the text from the punctuations 
def cleanpunc(sentence):
    cleaned = re.sub(r'[?|!|\'|"|#|{|}]',r'',str(sentence))
    cleaned = re.sub(r'[.|,|)|(|\|/]',r'',cleaned)
    return cleaned

In [62]:
import sys
import csv

csv.field_size_limit(sys.maxsize)
with open('/Users/ravishaggarwal/Desktop/new_data.csv') as input_file, open('/Users/ravishaggarwal/Desktop/clean_ops.csv', 'w') as output_file:
    r = csv.reader(input_file)
    row = next(r)
    counter = 0
    str1 = ''
    i = 0
    final_string = []
    for row in r:
        var1 = row[1].split()
        #print(var1)
        #counter = counter + 1
        #print(counter)
        #print(var1)
        a = set(var1)&EVM_OPCODE_SET
        #b = list(a)
        
        #print(b)
        #print(a,file = output_file)
        
                    
                    
        
                
            
            #print(sent)
            
        c = cleanpunc(a)
        print(''.join(c),file = output_file)
        
        #print(a)
        #print(b,file = output_file)
        #output_file.write(a)
        #counter = counter + 1
        #print(counter)
        
            
        #for i in var1:
            #print(i)
            #if(i in EVM_OPCODE_SET):
                #output_file.write(i)

In [48]:
#checking rows in csv 
#with open('/Users/ravishaggarwal/Desktop/new_data.csv') as file:
#    r = csv.reader(file)
#    row_count = sum(1 for row in r)

In [67]:
#bigrams matrix
import csv
import nltk
from collections import Counter
fo = open("/Users/ravishaggarwal/Desktop/clean_ops.csv")
fo1 = fo.readlines()
counter_sum = Counter()
for line in fo1:
    #tokens = nltk.word_tokenize(line)
    bigrams = list(nltk.bigrams(line.split()))
    bigramsC = Counter(bigrams)
    #tokensC = Counter(tokens)
    #both_counters = bigramsC + tokensC
    counter_sum += bigramsC
    #print(tokensC)
with open('/Users/ravishaggarwal/Desktop/unigrams_and_bigrams.csv', 'w', newline='') as csvfile:
    header = sorted(counter_sum, key=lambda x: str(type(x)))
    writer = csv.DictWriter(csvfile, fieldnames=header)
    writer.writeheader()
    for line in fo1:
          #tokens = nltk.word_tokenize(line)
          bigrams = list(nltk.bigrams(line.split()))
          bigramsC = Counter(bigrams)
          #tokensC = Counter(tokens)
          #both_counters = bigramsC + tokensC
          cs = dict(counter_sum)
          bc = dict(bigramsC)
          row = {}
          for element in list(cs):
                if element in list(bc):
                  row[element] = bc[element]
                else:
                  row[element] = 0
          writer.writerow(row)
    

In [83]:
import csv
with open("/Users/ravishaggarwal/Desktop/unigrams_and_bigrams.csv", "r") as f:
    reader = csv.DictReader(f)
    fieldnames = reader.fieldnames
    row_count = sum(1 for row in reader)

In [84]:
fieldnames

1457

In [85]:
row_count

4675929

In [89]:
import csv
afile = open("/Users/ravishaggarwal/Desktop/unigrams_and_bigrams.csv", 'r+')
csvReader1 = csv.reader(afile)
for i in range(3):
    print (csvReader1.__next__())

["('PUSH2', 'CALLDATACOPY')", "('CALLDATACOPY', 'CALLDATASIZE')", "('CALLDATASIZE', 'SUB')", "('SUB', 'DUP1')", "('DUP1', 'RETURN')", "('RETURN', 'CALLVALUE')", "('CALLVALUE', 'CALLCODE')", "('CALLCODE', 'GAS')", "('GAS', 'PUSH1')", "('PUSH1', 'PUSH20')", "('PUSH32', 'CALLER')", "('CALLER', 'SWAP4')", "('SWAP4', 'RETURN')", "('RETURN', 'STOP')", "('STOP', 'DUP6')", "('DUP6', 'OR')", "('OR', 'LOG1')", "('LOG1', 'CODECOPY')", "('CODECOPY', 'PUSH23')", "('PUSH23', 'JUMPI')", "('JUMPI', 'PUSH4')", "('PUSH4', 'DUP7')", "('DUP7', 'DUP1')", "('DUP1', 'JUMPDEST')", "('JUMPDEST', 'DUP3')", "('DUP3', 'DUP2')", "('DUP2', 'DUP8')", "('DUP8', 'DUP5')", "('DUP5', 'POP')", "('POP', 'PUSH1')", "('PUSH1', 'CALLDATALOAD')", "('CALLDATALOAD', 'DUP9')", "('DUP9', 'JUMP')", "('JUMP', 'CALLDATASIZE')", "('SUB', 'EQ')", "('EQ', 'NOT')", "('NOT', 'EXP')", "('EXP', 'GT')", "('GT', 'ADD')", "('ADD', 'ISZERO')", "('ISZERO', 'STATICCALL')", "('STATICCALL', 'SSTORE')", "('SSTORE', 'SWAP3')", "('SWAP3', 'DIV')", "(